In [0]:
print("Healthy")

In [0]:
from re import search
from sqlalchemy import create_engine
import duckdb

import pandas as pd 
import numpy as np 
import mysql.connector


pd.set_option('display.max_columns', None)


import warnings
warnings.filterwarnings('ignore')

import mysql.connector
import json
with open('/Workspace/Credentials/db_data.json', 'r') as fp:
    data = json.load(fp)

Dumpdb = mysql.connector.connect(
  host=data['Dumpdb']['host'],
  user=data['Dumpdb']['user'],
  passwd=data['Dumpdb']['passwd'],
  database = data['Dumpdb']['database']
)


Mifosdb = mysql.connector.connect(
  host=data['mifos']['host'],
  user=data['mifos']['user'],
  passwd=data['mifos']['passwd'],
  database = data['mifos']['database']
)


vbizdb = mysql.connector.connect(
     host=data['vbizdb']['host'],
  user=data['vbizdb']['user'],
  passwd=data['vbizdb']['passwd'],
  database = data['vbizdb']['database']
)


host = data['redshift']['host']
user = data['redshift']['user']
passwd = data['redshift']['passwd']
database = data['redshift']['database']

conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")

pd.set_option('display.float_format', lambda x: '%.2f' % x)

from datetime import datetime, timedelta
today = datetime.today().strftime('%Y-%m-%d')
yesterday =  (datetime.today() - timedelta(days = 1)).strftime('%Y-%m-%d')
print(today)
print(yesterday)


last_2_wks = datetime.today() - timedelta(days = 14)
last_2_wks = last_2_wks.strftime('%Y-%m-%d')
print('------------------------------------')
print(last_2_wks)

print('\n')
now = datetime.today().strftime('%Y-%m-%d %H:%M:%S')

last_30_mins = (datetime.today() - timedelta(days = 1)).strftime('%Y-%m-%d %H:%M:%S')
trunc_last_30_mins = (datetime.today() - timedelta(days = 1)).strftime('%Y-%m-%d %H:%M')
print(last_30_mins, 'to', now)

In [0]:
print(datetime.today().strftime('%Y-%m-%d %H:%M:%S'))

In [0]:
lastr = pd.read_sql_query(f'''
                select 
                -- *
                max(transaction_id) as ltr,
                max(transaction_date) as last_date
                from dwh_all_transactions
                -- order by transaction_date
                -- limit 100
                ''', conn)

ltr = (lastr['ltr'][0], lastr['last_date'][0])
print(ltr[0], '-', ltr[1])

lastr

In [0]:
# NOTE - to crosscheck reversal status btw Jan-2020 to Nov-2020 using (is_reversed_by_contra_entry) column on msat

## All transactions ETL from MIFOS

In [0]:
Mifosdb = mysql.connector.connect(
  host=data['mifos']['host'],
  user=data['mifos']['user'],
  passwd=data['mifos']['passwd'],
  database = data['mifos']['database']
)

miftr = pd.read_sql_query(f'''
                select
                st.id as transaction_id,
                
                ifnull(mc.id, mg.id) as client_id,
                ifnull(replace(mc.display_name, 'null', ''), mg.display_name) as client_name,
                st.savings_account_id as account_id,
                msa.account_no,
                msp.name as product_name,
                st.amount, 
                st.overdraft_amount_derived as overdraft_amount, 
                st.running_balance_derived as running_balance, 
                st.cumulative_balance_derived as cumulative_balance, 
                st.created_date transaction_date,   
                case st.is_reversed_by_contra_entry
                        when '0' then 'No'
                        when '1' then 'Yes' end as is_reversed,
                case st.is_loan_disbursement
                        when 0 then 'No'
                        when 1 then 'Yes' end as is_transaction_loan_disbursement,

                map.username as transaction_initializer_username, 
                CONCAT(map.firstname, ' ', map.lastname) as transaction_initializer,
                
                st.transaction_type_enum, 
                renum.enum_value as transaction_type,
                mpd.account_number as third_party_account_no, 
                mpd.check_number as channel, 
                mpd.receipt_number session_id, 
                mpd.bank_number, 
                mpd.routing_code,
                
                mpt.value as partners_description,
                mtr.latitude, 
                mtr.longitude, 
                mtr.notes transaction_notes, 
                mtr.remarks transaction_remarks,
                mtr.transaction_brand_name as third_party_name                
                
                from m_savings_account_transaction st
                left join m_savings_account msa on msa.id = st.savings_account_id
                left join m_savings_product msp on msp.id = msa.product_id
                left join m_client mc on mc.id = msa.client_id
                left join m_group mg on mg.id = msa.group_id
                left join (SELECT
                            rv.enum_name,
                            rv.enum_id,
                            rv.enum_message_property,
                            rv.enum_value
                            FROM r_enum_value rv
                            WHERE rv.enum_name = 'savings_transaction_type_enum') renum on renum.enum_id = st.transaction_type_enum
                left join m_appuser map on map.id = st.appuser_id
                left JOIN m_payment_detail mpd ON mpd.id = st.payment_detail_id
                left JOIN m_payment_type mpt ON mpt.id = mpd.payment_type_id
                left join m_transaction_request mtr on mtr.transaction_id = st.id
                where left(transaction_date, 4) >= '2019'
                and  st.id > {ltr[0]}
                LIMIT 50000
                ''', Mifosdb)

miftr['transaction_id'] = miftr['transaction_id'].astype(str)


dump = pd.read_sql_query(f'''
            select
            resource_id as transaction_id,
            response_code,
            partners
            from MM_BANK_SETTLEMENTS
            where resource_id in ({','.join([str(x) for x in miftr['transaction_id'].tolist()])})
            and partners != 'SAVI'
            ''', Dumpdb)

    
full = pd.merge(miftr, dump, on ='transaction_id', how = 'left')

full['transaction_id'] = full['transaction_id'].astype('int64')
full['load_date'] = datetime.today().strftime('%Y-%m-%d %H:%M:%S')


In [0]:
full.head()

In [0]:
len(full)

In [0]:
#%%time

#full.to_sql('dwh_all_transactions', conn, index = False, if_exists = 'append', chunksize = 10000, method = 'multi')

#print(f'ETL Successful! as at {datetime.today()}')
#print('\n\n')


In [0]:
%%time

chunk_size = 10000
num_rows = len(full)

# Create a SQLAlchemy engine
host = data['redshift']['host']
user = data['redshift']['user']
passwd = data['redshift']['passwd']
database = data['redshift']['database']

conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")

# Calculate the number of iterations needed
num_iterations = (num_rows // chunk_size) + 1 if num_rows % chunk_size != 0 else num_rows // chunk_size

# Loop through the DataFrame in chunks and insert into the database
for i in range(num_iterations):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size if (i + 1) * chunk_size < num_rows else num_rows
    
    # Extract the chunk of data
    chunk_df = full.iloc[start_idx:end_idx]
    
    # Use df.to_sql() to insert the chunk into the database
    chunk_df.to_sql("dwh_all_transactions", conn, index = False, if_exists = 'append', chunksize = 5000, method = 'multi')


print(f'ETL Successful! as at {datetime.today()}')
print('\n\n')

## Paycode / Cardless transactions

In [0]:
%%time
paycode = pd.read_sql(f'''
                select 
                id,
                resource_id as transaction_id,
                reference,
                account_number,
                account_id,
                amount,
                transaction_ref,
                -- response_message,
                response_code,
                channel,
                partners,
                posting_date as transaction_date,
                resource_rvs_id,
                posting_rvs_date,
                modified as modified_date,
                paycode_withdrawal_resource_id,
                paycode_trans_id,
                is_set_for_paycode_reversal
                paycode_withdrawal_reversal_resource_id,
                now() as load_date
                from MM_BANK_SETTLEMENTS
                where partners = 'PAYCODE'
                ''', Dumpdb)

paycode['resource_rvs_id'] = paycode['resource_rvs_id'].fillna(0).astype('int64')
paycode.head()

In [0]:
%%time
#paycode.to_sql("dwh_paycode_transactions", conn, index = False, if_exists = 'replace', chunksize = 10000, method = 'multi')

In [0]:
%%time

chunk_size = 10000
num_rows = len(paycode)

# Create a SQLAlchemy engine
conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")

# Calculate the number of iterations needed
num_iterations = (num_rows // chunk_size) + 1 if num_rows % chunk_size != 0 else num_rows // chunk_size

# Loop through the DataFrame in chunks and insert into the database
for i in range(num_iterations):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size if (i + 1) * chunk_size < num_rows else num_rows
    
    # Extract the chunk of data
    chunk_df = paycode.iloc[start_idx:end_idx]
    
    # Use df.to_sql() to insert the chunk into the database
    chunk_df.to_sql("dwh_paycode_transactions", conn, index = False, if_exists = 'append', chunksize = 5000, method = 'multi')


print(f'ETL Successful! as at {datetime.today()}')
print('\n\n')

Mifosdb = mysql.connector.connect(
  host=data['mifos']['host'],
  user=data['mifos']['user'],
  passwd=data['mifos']['passwd'],
  database = data['mifos']['database']
)

df = pd.read_sql_query(f"""
    select
    dac.client_id,
    dac.client_name,
    dac.client_status,
    dac.country,
    dac.client_gender,
    dac.client_type,
    dac.client_tier,
    dac.client_category ,
    daa.account_no,
    dab.approvedon_date,
    dab.closedon_date ,
    dab.balance,
    dab.product_id,
    daa.product_name,
    dpt.short_name,
    dpt.description
from
    dwh_all_clients dac
left join
  dwh_all_accounts daa
on
    dac.client_id = daa.client_id
left join
  dwh_account_balances dab
on
    daa.account_no = dab.account_no
left join
  dwh_product_types dpt
on
    daa.product_id = dpt.product_id
where
    UPPER(dac.lastname) = 'OMOTAYO';
    """, conn)

## Check for duplicates on all transactions table

%%time
ch = """
select 
transaction_id,
count(*)
from dwh_all_transactions
group by 1
having count(transaction_id ) > 1
LIMIT 300000
"""

df_ch = pd.read_sql_query(ch, conn)
print(len(df_ch))



if len(df_ch) > 0:

    dups = ','.join([str(x) for x in df_ch['transaction_id'].tolist()])

    ac = f"""
    select 
    distinct *
    from dwh_all_transactions
    where transaction_id in ({dups})
    """

    df = pd.read_sql_query(ac, conn)

    # delete rows from table on redshift SQL server
    sql = f'''
        DELETE FROM dwh_all_transactions
        WHERE "transaction_id" in ({dups})
       '''

    with conn.begin() as engine:
        engine.execute(sql)

    df.to_sql('dwh_all_transactions', conn, index=False, if_exists='append', chunksize=10000, method='multi')
    
else:
    print('no airtime duplicates found!!!')
